In [1]:
import sys,os
import pandas as pd
import random
import numpy as np

In [2]:
module_path=os.path.abspath('/root/NLP_warehouse/reconstruct/basictask/')
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/root/miniconda3/envs/nlp_sr/lib/python36.zip', '/root/miniconda3/envs/nlp_sr/lib/python3.6', '/root/miniconda3/envs/nlp_sr/lib/python3.6/lib-dynload', '/root/.local/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/root/NLP_warehouse/reconstruct/basictask']


In [3]:
from tasks import sts
from evaluation import stsEvaluator
from readers.sts import getExamples,convert_sentences_to_features,convert_examples_to_features

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
data_folder='../../datasets/lcqmc'
label2id={"0":0,"1":1}
train_examples=getExamples(os.path.join(data_folder,'lcqmc_train.tsv'),label2id=label2id,isCL=False,filter_heads=True)

2021-08-20 14:10:41 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-20 14:10:41 - INFO - getExamples - 57 : *****************************Logging some train examples*****************************
2021-08-20 14:10:41 - INFO - getExamples - 58 : Total train nums is : 238766
2021-08-20 14:10:41 - INFO - getExamples - 61 : 奥比岛的精灵岛在哪里？	奥比岛在哪里买精灵蛋	0
2021-08-20 14:10:41 - INFO - getExamples - 61 : 中国最好看的女人	中国最好看的电视剧	0
2021-08-20 14:10:41 - INFO - getExamples - 61 : 车玻璃贴的纸叫什么纸	暖贴是什么意思？贴吧上的	0
2021-08-20 14:10:41 - INFO - getExamples - 61 : 孕妇不能吃什么？	孕妇不能吃什么啊	1
2021-08-20 14:10:41 - INFO - getExamples - 61 : 苹果7什么时候上市	苹果5什么时候上市	0


In [5]:
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
sts_model=sts(model_path=model_path,device='cpu')

2021-08-20 14:10:41 - INFO - __init__ - 77 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-20 14:10:41 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-20 14:10:41 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-20 14:10:41 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-20 14:10:43 - INFO - from_pretrained - 124 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt
2021-08-20 14:10:43 - INFO - __init__ - 84 : Softmax loss: #Vectors concatenated: 3
2021-08-20 14:10:43 - INFO - __init__ - 85 : After pooling, each sentence embedding has dim: 768


In [6]:
sts_model.encode(["这是什么植物？谢谢"]).shape

(1, 768)

In [7]:
dev_examples=getExamples(os.path.join(data_folder,'lcqmc_dev.tsv'),label2id=label2id,filter_heads=True,mode='dev',isCL=False)
test_examples=getExamples(os.path.join(data_folder,'lcqmc_test.tsv'),label2id=label2id,filter_heads=True,mode='test',isCL=False)
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

2021-08-20 14:10:43 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-20 14:10:43 - INFO - getExamples - 57 : *****************************Logging some dev examples*****************************
2021-08-20 14:10:43 - INFO - getExamples - 58 : Total dev nums is : 8802
2021-08-20 14:10:43 - INFO - getExamples - 61 : 为什么原子是化学变化中最小的粒子	说原子是化学变化中的最小粒子根据	1
2021-08-20 14:10:43 - INFO - getExamples - 61 : 广东科学中心怎么去？交通指南	从芳村怎么去广东科学中心？	0
2021-08-20 14:10:43 - INFO - getExamples - 61 : 微信朋友圈里的消息怎么看？	微信朋友圈里别人的发表怎样复制	0
2021-08-20 14:10:43 - INFO - getExamples - 61 : 我的世界咋卡村庄	我的世界怎么找村庄	0
2021-08-20 14:10:43 - INFO - getExamples - 61 : 地瓜怎么吃	怎么煮地瓜	0
2021-08-20 14:10:43 - INFO - getExamples - 44 : Heads like : text_a	text_b	label

2021-08-20 14:10:43 - INFO - getExamples - 57 : *****************************Logging some test examples*****************************
2021-08-20 14:10:43 - INFO - getExamples - 58 : Total test nums is : 12500
2021-08-20 14:10:43 - INFO - getExamples - 61 : 信

In [8]:
dev_sentences=[example.text_list for example in dev_examples]
dev_labels=[example.label for example in dev_examples]

In [9]:
print(dev_sentences[0],dev_labels[0])

['开初婚未育证明怎么弄？', '初婚未育情况证明怎么开？'] 1


In [10]:
sentences1_list=[sen[0] for sen in dev_sentences]
sentences2_list=[sen[1] for sen in dev_sentences]
evaluator=stsEvaluator(sentences1=sentences1_list,sentences2=sentences2_list,batch_size=64,write_csv=False,scores=dev_labels)

In [ ]:
sts_model.fit(train_dataloader=train_dataloader,epochs=5,evaluator=evaluator,output_path="/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/")

2021-08-20 14:10:43 - INFO - fit - 145 : 一个epoch 下，每隔1492个step会输出一次loss，每隔3731个step会评估一次模型


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-08-20 14:10:46 - INFO - __call__ - 72 : EmbeddingSimilarityEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:


Batches:   0%|          | 0/138 [00:00<?, ?it/s]

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

2021-08-20 14:11:39 - INFO - __call__ - 104 : Cosine-Similarity :	Pearson: 0.5057	Spearman: 0.5758
2021-08-20 14:11:39 - INFO - __call__ - 106 : Manhattan-Distance:	Pearson: 0.5592	Spearman: 0.5813
2021-08-20 14:11:39 - INFO - __call__ - 108 : Euclidean-Distance:	Pearson: 0.5622	Spearman: 0.5854
2021-08-20 14:11:39 - INFO - __call__ - 110 : Dot-Product-Similarity:	Pearson: 0.2944	Spearman: 0.2848
2021-08-20 14:11:41 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-20 14:11:42 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-20 14:11:42 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-20 14:11:42 - INFO - fit - 221 : In epoch 0, training_step 0, the eval score is 0.5758478173416628, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xh